In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

VER = 175
USE = 'orders'

PART = 3

In [2]:
import pandas as pd, cudf, numpy as np, gc
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt

In [3]:
df = cudf.read_parquet('../../data/infer_data/test.parquet')
df.ts = (df.ts//1000).astype('int32')

df = df.sort_values(['session','ts'])
df = df.groupby('session').agg('last').reset_index()
df = df[['session','aid']]
df.columns = ['user','last']
print( df.shape )
df.head()

(1671803, 2)


,user,last
0,12899779,59625
1,12899780,1142000
2,12899781,918667
3,12899782,1007613
4,12899783,1817895


In [4]:
users = pd.read_parquet(f'../../data/infer_with_features/train_v{VER}_LB_{PART}.pqt')
print( users.shape )
users.head()

(83590150, 154)


,user,item,pos,wgt,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5,buy_ratio_0_6,buy_ratio_1_6,buy_ratio_2_6,buy_ratio_3_6,buy_ratio_4_6,buy_ratio_5_6,buy_ratio_6_6,count_item_0_6,count_item_1_6,count_item_2_6,count_item_3_6,count_item_4_6,count_item_5_6,count_item_6_6,count_user_0_6,count_user_1_6,count_user_2_6,count_user_3_6,count_user_4_6,count_user_5_6,count_user_6_6,item_day_std_6,user_day_std_6,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3,repeat3,buy_ratio_std3,hour_std3,day_std3,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3,buy_ratio2,count_item2,unique_item2,hour_mean2,day_mean2,repeat2,buy_ratio_std2,hour_std2,day_std2,ts_max7,ts_min7,ts_diff7,z1,z2,z3,z4,z5,z6,buy_ratio7,count_item7,count_user7,hour_mean7,day_mean7,repeat7,buy_ratio_std7,hour_std7,day_std7,prev7,next7,orders7,carts7,order_repeat7,cart_repeat7,buy_ratio8,count_item8,count_user8,hour_mean8,day_mean8,repeat8,buy_ratio_std8,hour_std8,day_std8,prev8,next8,orders8,carts8,order_repeat8,cart_repeat8,buy_ratio9,count_item9,count_user9,hour_mean9,day_mean9,repeat9,buy_ratio_std9,hour_std9,day_std9,prev9,next9,orders9,carts9,order_repeat9,cart_repeat9,buy_ratio10,count_item10,unique_item10,hour_mean10,day_mean10,repeat10,buy_ratio_std10,hour_std10,day_std10
0,12899779,59625,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.414214,-1.0,0.071773,-1.0,-1.0,1.414214,1.414214,1.414214,1.414214,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.000000,4.0,4.0,1.000000,0.000000,NaN,NaN,0.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.377964,0.377964,0.000000,12,11,56063.832031,315263.84375,1.090909,0.000000,18902.294922,174509.890625,0.000000,1.0,1.0,79200.000000,338400.0000,1.0,-1.000000,-1.000000,-1.000000,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,1,1,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,0,0,0,1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,0.000000,2.0,2.0,64435.000000,150835.00000,1.000000,0.00000,8051.117676,114136.937500,2.0,2.0,0.0,0.0,-1.0,-1.0,0.000000,6.0,5.0,66307.000000,325507.00000,1.200000,0.000000,7429.592773,169454.437500,5.0,5.0,0.0,0.0,-1.0,-1.0,0.000000,1.0,1.0,9485.000000,95885.00000,1.000000,-1.000000,-1.000000,-1.000000,1.0,1.0,0.0,0.0,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12899779,737445,1,3.0,1.0,-3.0,-3.0,-3.0,2.0,-3.0,0.850000,3.0,1.500000,8.0,8.0,-3.000000,-3.000000,-3.000000,0.850000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,2.0,1.0,0.022444,401.0,258.0,1.554264,0.148307,NaN,NaN,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,4.0,7.0,6.0,3.0,3.0,3.0,9.0,4.0,4.0,4.0,3.0,3.0,2.0,7.0,2.380476,2.380476,0.016234,924,575,50592.035156,303620.59375,1.606956,0.126442,19655.660156,170439.515625,0.028571,35.0,25.0,50927.429688,349624.5625,1.4,0.169031,21019.531250,161340.265625,15.0,12.0,0.0,1.0,-1.0,1.0,0.0,1,1,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,0,0,0,0.954863,-1.0,-1.0,-1.0,-1.0,-1.0,0.021201,283.0,180.0,51423.164062,294747.53125,1.572222,0.14431,19882.255859,162015.390625,142.0,133.0,0.0,6.0,-1.0,1.0,0.019417,412.0,255.0,60896.785156,322403.59375,1.615686,0.138155,19028.656250,166305.156250,197.0,183.0,0.0,8.0,-1.0,1.0,0.019231,312.0,215.0,36553.199219,271937.81250,1.451163,0.137556,9282.897461,162549.218750,148.0,134.0,0.0,6.0,-1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12899779,1790770,2,2.5,2.0,-3.0,-3.0,-3.0,0.5,-3.0,0.050000,2.5,0.900000,4.0,4.0,-3.000000,-3.000000,-3.000000,0.050000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,2.0,0.5,1.0,0.200000,20.0,13.0,1.538462,0.523148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.111111,36,26,43221.527344,326421.53125,1.384615,0.398410,16429.857422,183993.187500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [5]:
%%time
users2 = users[['user','item']].copy()
users2 = users2.merge(df.to_pandas(),on='user',how='left')
print( users2.shape )
users2.head()

(83590150, 3)
CPU times: user 5.72 s, sys: 8.56 s, total: 14.3 s
Wall time: 7.11 s


,user,item,last
0,12899779,59625,59625
1,12899779,737445,59625
2,12899779,1790770,59625
3,12899779,448688,59625
4,12899779,469285,59625


In [6]:
df = cudf.read_parquet('../../data/infer_data/test.parquet')
df.ts = (df.ts//1000).astype('int32')

df = df.sort_values(['session','ts'])
df = df.groupby('session').agg('first').reset_index()
df = df[['session','aid']]
df.columns = ['user','first']
print( df.shape )
df.head()

(1671803, 2)


,user,first
0,12899779,59625
1,12899780,1142000
2,12899781,141736
3,12899782,1669402
4,12899783,255297


In [7]:
%%time
users2 = users2.merge(df.to_pandas(),on='user',how='left')
print( users2.shape )
users2.head()

(83590150, 4)
CPU times: user 5.91 s, sys: 1.45 s, total: 7.36 s
Wall time: 7.35 s


,user,item,last,first
0,12899779,59625,59625,59625
1,12899779,737445,59625,59625
2,12899779,1790770,59625,59625
3,12899779,448688,59625,59625
4,12899779,469285,59625,59625


In [8]:
df = cudf.read_parquet('../../data/infer_data/test.parquet')
df.ts = (df.ts//1000).astype('int32')

df = df.sort_values(['session','ts']).reset_index(drop=True)

df['mx'] = df.groupby('session').ts.transform('max')
df['mx'] = df.mx - df.ts
df = df.loc[df.mx > 60*60*24]

df = df.groupby('session').agg('last').reset_index()
df = df[['session','aid']]
df.columns = ['user','last2']
print( df.shape )
df.head()

(95576, 2)


,user,last2
0,12899781,199008
1,12899783,607638
2,12899784,1036375
3,12899795,207754
4,12899798,211030


In [9]:
%%time
users2 = users2.merge(df.to_pandas(),on='user',how='left')
users2['last2'] = users2['last2'].fillna(users2['last']).astype('int32')
print( users2.shape )
users2.head()

(83590150, 5)
CPU times: user 6.57 s, sys: 1.94 s, total: 8.5 s
Wall time: 8.5 s


,user,item,last,first,last2
0,12899779,59625,59625,59625,59625
1,12899779,737445,59625,59625,59625
2,12899779,1790770,59625,59625,59625
3,12899779,448688,59625,59625,59625
4,12899779,469285,59625,59625,59625


In [10]:
users2['last'].isna().sum()

0

In [11]:
users2['last2'].isna().sum()

0

In [12]:
users2['first'].isna().sum()

0

# Matrix Factorization

In [13]:
%%time
x = users2['first'].values
y = users2['last'].values

CPU times: user 34 µs, sys: 11 µs, total: 45 µs
Wall time: 50.3 µs


In [14]:
embed = np.load('../../data/item_user_features/item_embed_32_LB.npy')
z = np.sqrt( np.sum(embed * embed,axis=1) )
embed = embed/ np.reshape(z,(-1,1))
embed.shape
#(1855603, 32)

(1855603, 32)

In [15]:
%%time
e1 = embed[x,]
e2 = embed[y,]

CPU times: user 3.86 s, sys: 1.85 s, total: 5.71 s
Wall time: 5.71 s


In [16]:
%%time
z = np.sum( e1*e2,axis=1 )

CPU times: user 5.3 s, sys: 1.1 s, total: 6.4 s
Wall time: 6.4 s


In [17]:
users2['z10'] = z

In [18]:
%%time
x = users2['last2'].values
#y = users2['last'].values

CPU times: user 36 µs, sys: 11 µs, total: 47 µs
Wall time: 53.6 µs


In [19]:
%%time
e1 = embed[x,]
#e2 = embed[y,]

CPU times: user 2.36 s, sys: 603 ms, total: 2.96 s
Wall time: 2.96 s


In [20]:
%%time
z = np.sum( e1*e2,axis=1 )

CPU times: user 5.28 s, sys: 1.13 s, total: 6.41 s
Wall time: 6.4 s


In [21]:
users2['z11'] = z

# Add Item Features

In [22]:
df = pd.read_parquet('../../data/item_user_features/item40_LB.pqt')
for c in df.columns:
    if str(df[c].dtype)=='int32':
        print(c, df[c].dtype)
        df[c] = df[c].astype('float32')
df.head()

count_item11 int32
count_user11 int32


,buy_ratio11,count_item11,count_user11,hour_mean11,day_mean11,repeat11,buy_ratio_std11,hour_std11,day_std11
aid,,,,,,,,,
1,1.000000,1.0,1.0,56602.000000,575002.00000,1.000000,-1.000000,-1.000000,-1.000000
3,1.169643,224.0,157.0,52163.367188,289763.37500,1.426752,0.376159,15203.037109,144851.250000
4,1.000000,8.0,8.0,58075.250000,241675.25000,1.000000,0.000000,15992.715820,135522.828125
8,1.000000,1.0,1.0,49243.000000,567643.00000,1.000000,-1.000000,-1.000000,-1.000000
10,1.333333,3.0,2.0,65387.667969,295787.65625,1.500000,0.577350,4047.108643,53823.906250


In [23]:
%%time
users2 = users2.merge(df, left_on='item', right_index=True, how='left')
print( users2.shape )
users2.head()

(83590150, 16)
CPU times: user 15.3 s, sys: 1.93 s, total: 17.2 s
Wall time: 17.2 s


,user,item,last,first,last2,z10,z11,buy_ratio11,count_item11,count_user11,hour_mean11,day_mean11,repeat11,buy_ratio_std11,hour_std11,day_std11
0,12899779,59625,59625,59625,59625,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12899779,737445,59625,59625,59625,1.0,1.0,1.0,14.0,10.0,41221.214844,288078.34375,1.4,0.0,22480.066406,134324.906250
2,12899779,1790770,59625,59625,59625,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12899779,448688,59625,59625,59625,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12899779,469285,59625,59625,59625,1.0,1.0,1.0,2.0,2.0,57165.000000,532365.00000,1.0,0.0,9224.915039,51869.109375


In [24]:
df = pd.read_parquet('.../../data/item_user_features/item41_LB.pqt')
for c in df.columns:
    if str(df[c].dtype)=='int32':
        print(c, df[c].dtype)
        df[c] = df[c].astype('float32')
df.head()

count_item12 int32
count_user12 int32


,buy_ratio12,count_item12,count_user12,hour_mean12,day_mean12,repeat12,buy_ratio_std12,hour_std12,day_std12
aid,,,,,,,,,
3,1.136364,22.0,17.0,48736.136719,484663.40625,1.294118,0.35125,17377.468750,74483.609375
21,1.000000,2.0,1.0,57269.500000,316469.50000,2.000000,0.00000,47.376156,47.376156
24,1.000000,1.0,1.0,59457.000000,145857.00000,1.000000,-1.00000,-1.000000,-1.000000
25,1.000000,1.0,1.0,59396.000000,59396.00000,1.000000,-1.00000,-1.000000,-1.000000
39,1.000000,2.0,1.0,27360.000000,27360.00000,2.000000,0.00000,4.242640,4.242640


In [25]:
%%time
users2 = users2.merge(df, left_on='item', right_index=True, how='left')
print( users2.shape )
users2.head()

(83590150, 25)
CPU times: user 16 s, sys: 2.3 s, total: 18.3 s
Wall time: 18.3 s


,user,item,last,first,last2,z10,z11,buy_ratio11,count_item11,count_user11,hour_mean11,day_mean11,repeat11,buy_ratio_std11,hour_std11,day_std11,buy_ratio12,count_item12,count_user12,hour_mean12,day_mean12,repeat12,buy_ratio_std12,hour_std12,day_std12
0,12899779,59625,59625,59625,59625,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12899779,737445,59625,59625,59625,1.0,1.0,1.0,14.0,10.0,41221.214844,288078.34375,1.4,0.0,22480.066406,134324.906250,1.0,1.0,1.0,39966.0,558366.0,1.0,-1.0,-1.0,-1.0
2,12899779,1790770,59625,59625,59625,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12899779,448688,59625,59625,59625,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,40851.0,472851.0,1.0,-1.0,-1.0,-1.0
4,12899779,469285,59625,59625,59625,1.0,1.0,1.0,2.0,2.0,57165.000000,532365.00000,1.0,0.0,9224.915039,51869.109375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
COLS = [f'{x.replace("12","13")}' for x in df.columns]
df.columns = COLS
print( COLS )

['buy_ratio13', 'count_item13', 'count_user13', 'hour_mean13', 'day_mean13', 'repeat13', 'buy_ratio_std13', 'hour_std13', 'day_std13']


In [27]:
%%time
users2 = users2.merge(df, left_on='last', right_index=True, how='left')
print( users2.shape )
users2.head()

(83590150, 34)
CPU times: user 7.78 s, sys: 2.49 s, total: 10.3 s
Wall time: 10.3 s


,user,item,last,first,last2,z10,z11,buy_ratio11,count_item11,count_user11,hour_mean11,day_mean11,repeat11,buy_ratio_std11,hour_std11,day_std11,buy_ratio12,count_item12,count_user12,hour_mean12,day_mean12,repeat12,buy_ratio_std12,hour_std12,day_std12,buy_ratio13,count_item13,count_user13,hour_mean13,day_mean13,repeat13,buy_ratio_std13,hour_std13,day_std13
0,12899779,59625,59625,59625,59625,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12899779,737445,59625,59625,59625,1.0,1.0,1.0,14.0,10.0,41221.214844,288078.34375,1.4,0.0,22480.066406,134324.906250,1.0,1.0,1.0,39966.0,558366.0,1.0,-1.0,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12899779,1790770,59625,59625,59625,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12899779,448688,59625,59625,59625,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,40851.0,472851.0,1.0,-1.0,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12899779,469285,59625,59625,59625,1.0,1.0,1.0,2.0,2.0,57165.000000,532365.00000,1.0,0.0,9224.915039,51869.109375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Add User Features

In [28]:
df = pd.read_parquet('../../data/item_user_features/user20_LB.pqt')
for c in df.columns:
    if str(df[c].dtype)=='int32':
        print(c, df[c].dtype)
        df[c] = df[c].astype('float32')
df.head()

count_item14 int32
unique_item14 int32


,buy_ratio14,count_item14,unique_item14,hour_mean14,day_mean14,repeat14,buy_ratio_std14,hour_std14,day_std14
session,,,,,,,,,
12899779,0.000000,1.0,1.0,79200.000000,338400.00000,1.000000,-1.000000,-1.000000,-1.000000
12899780,0.000000,5.0,4.0,79291.601562,338491.59375,1.250000,0.000000,62.875275,62.875275
12899781,0.000000,3.0,2.0,69684.000000,69684.00000,1.500000,0.000000,68.234886,68.234886
12899782,0.457143,70.0,38.0,64777.812500,398034.96875,1.842105,0.695445,14570.271484,27593.218750
12899783,0.000000,1.0,1.0,50740.000000,50740.00000,1.000000,-1.000000,-1.000000,-1.000000


In [29]:
%%time
users2 = users2.merge(df, left_on='user', right_index=True, how='left')
print( users2.shape )
users2.head()

(83590150, 43)
CPU times: user 8.03 s, sys: 2.86 s, total: 10.9 s
Wall time: 10.9 s


,user,item,last,first,last2,z10,z11,buy_ratio11,count_item11,count_user11,hour_mean11,day_mean11,repeat11,buy_ratio_std11,hour_std11,day_std11,buy_ratio12,count_item12,count_user12,hour_mean12,day_mean12,repeat12,buy_ratio_std12,hour_std12,day_std12,buy_ratio13,count_item13,count_user13,hour_mean13,day_mean13,repeat13,buy_ratio_std13,hour_std13,day_std13,buy_ratio14,count_item14,unique_item14,hour_mean14,day_mean14,repeat14,buy_ratio_std14,hour_std14,day_std14
0,12899779,59625,59625,59625,59625,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0
1,12899779,737445,59625,59625,59625,1.0,1.0,1.0,14.0,10.0,41221.214844,288078.34375,1.4,0.0,22480.066406,134324.906250,1.0,1.0,1.0,39966.0,558366.0,1.0,-1.0,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0
2,12899779,1790770,59625,59625,59625,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0
3,12899779,448688,59625,59625,59625,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,40851.0,472851.0,1.0,-1.0,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0
4,12899779,469285,59625,59625,59625,1.0,1.0,1.0,2.0,2.0,57165.000000,532365.00000,1.0,0.0,9224.915039,51869.109375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0


In [30]:
df = pd.read_parquet('../../data/item_user_features/user21_LB.pqt')
for c in df.columns:
    if str(df[c].dtype)=='int32':
        print(c, df[c].dtype)
        df[c] = df[c].astype('float32')
df.head()

count_item15 int32
unique_item15 int32


,buy_ratio15,count_item15,unique_item15,hour_mean15,day_mean15,repeat15,buy_ratio_std15,hour_std15,day_std15
session,,,,,,,,,
12899782,1.333333,24.0,20.0,65109.875000,407109.87500,1.2,0.481543,13592.392578,19700.259766
12899786,1.000000,1.0,1.0,79217.000000,338417.00000,1.0,-1.000000,-1.000000,-1.000000
12899787,1.000000,3.0,1.0,79232.000000,338432.00000,3.0,0.000000,8.888194,8.888194
12899790,1.333333,3.0,2.0,79284.335938,338484.34375,1.5,0.577350,128.165253,128.165253
12899793,2.000000,3.0,3.0,42241.000000,387841.00000,1.0,0.000000,0.000000,0.000000


In [31]:
%%time
users2 = users2.merge(df, left_on='user', right_index=True, how='left')
print( users2.shape )
users2.head()

(83590150, 52)
CPU times: user 7.99 s, sys: 3.11 s, total: 11.1 s
Wall time: 11.1 s


,user,item,last,first,last2,z10,z11,buy_ratio11,count_item11,count_user11,hour_mean11,day_mean11,repeat11,buy_ratio_std11,hour_std11,day_std11,buy_ratio12,count_item12,count_user12,hour_mean12,day_mean12,repeat12,buy_ratio_std12,hour_std12,day_std12,buy_ratio13,count_item13,count_user13,hour_mean13,day_mean13,repeat13,buy_ratio_std13,hour_std13,day_std13,buy_ratio14,count_item14,unique_item14,hour_mean14,day_mean14,repeat14,buy_ratio_std14,hour_std14,day_std14,buy_ratio15,count_item15,unique_item15,hour_mean15,day_mean15,repeat15,buy_ratio_std15,hour_std15,day_std15
0,12899779,59625,59625,59625,59625,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12899779,737445,59625,59625,59625,1.0,1.0,1.0,14.0,10.0,41221.214844,288078.34375,1.4,0.0,22480.066406,134324.906250,1.0,1.0,1.0,39966.0,558366.0,1.0,-1.0,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12899779,1790770,59625,59625,59625,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12899779,448688,59625,59625,59625,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,40851.0,472851.0,1.0,-1.0,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12899779,469285,59625,59625,59625,1.0,1.0,1.0,2.0,2.0,57165.000000,532365.00000,1.0,0.0,9224.915039,51869.109375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# EDA Columns

In [32]:
for c in users2.columns:
    print(c,users2[c].dtype)

user int32
item int32
last int64
first int64
last2 int32
z10 float32
z11 float32
buy_ratio11 float32
count_item11 float32
count_user11 float32
hour_mean11 float32
day_mean11 float32
repeat11 float32
buy_ratio_std11 float32
hour_std11 float32
day_std11 float32
buy_ratio12 float32
count_item12 float32
count_user12 float32
hour_mean12 float32
day_mean12 float32
repeat12 float32
buy_ratio_std12 float32
hour_std12 float32
day_std12 float32
buy_ratio13 float32
count_item13 float32
count_user13 float32
hour_mean13 float32
day_mean13 float32
repeat13 float32
buy_ratio_std13 float32
hour_std13 float32
day_std13 float32
buy_ratio14 float32
count_item14 float32
unique_item14 float32
hour_mean14 float32
day_mean14 float32
repeat14 float32
buy_ratio_std14 float32
hour_std14 float32
day_std14 float32
buy_ratio15 float32
count_item15 float32
unique_item15 float32
hour_mean15 float32
day_mean15 float32
repeat15 float32
buy_ratio_std15 float32
hour_std15 float32
day_std15 float32


In [33]:
for c in users2.columns[:5]:
    print(c,', ',end='')

user , item , last , first , last2 , 

In [34]:
for c in users2.columns[5:]:
    print(c,', ',end='')
    users[c] = users2[c].values
del users2
gc.collect()

z10 , z11 , buy_ratio11 , count_item11 , count_user11 , hour_mean11 , day_mean11 , repeat11 , buy_ratio_std11 , hour_std11 , day_std11 , buy_ratio12 , count_item12 , count_user12 , hour_mean12 , day_mean12 , repeat12 , buy_ratio_std12 , hour_std12 , day_std12 , buy_ratio13 , count_item13 , count_user13 , hour_mean13 , day_mean13 , repeat13 , buy_ratio_std13 , hour_std13 , day_std13 , buy_ratio14 , count_item14 , unique_item14 , hour_mean14 , day_mean14 , repeat14 , buy_ratio_std14 , hour_std14 , day_std14 , buy_ratio15 , count_item15 , unique_item15 , hour_mean15 , day_mean15 , repeat15 , buy_ratio_std15 , hour_std15 , day_std15 , 

155

In [35]:
for c in users.columns:
    t = str(users[c].dtype)
    if (t=='int64'): 
        users[c] = users[c].astype('int32')
        print('convert',c,'to int32')
    if (t=='float64')|(t=='Int32')|(t=='Float32'):
        users[c] = users[c].astype('float32')
        print('convert',c,'to float32')

In [36]:
print( users.shape )
users.head()

(83590150, 201)


,user,item,pos,wgt,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5,buy_ratio_0_6,buy_ratio_1_6,buy_ratio_2_6,buy_ratio_3_6,buy_ratio_4_6,buy_ratio_5_6,buy_ratio_6_6,count_item_0_6,count_item_1_6,count_item_2_6,count_item_3_6,count_item_4_6,count_item_5_6,count_item_6_6,count_user_0_6,count_user_1_6,count_user_2_6,count_user_3_6,count_user_4_6,count_user_5_6,count_user_6_6,item_day_std_6,user_day_std_6,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3,repeat3,buy_ratio_std3,hour_std3,day_std3,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3,buy_ratio2,count_item2,unique_item2,hour_mean2,day_mean2,repeat2,buy_ratio_std2,hour_std2,day_std2,ts_max7,ts_min7,ts_diff7,z1,z2,z3,z4,z5,z6,buy_ratio7,count_item7,count_user7,hour_mean7,day_mean7,repeat7,buy_ratio_std7,hour_std7,day_std7,prev7,next7,orders7,carts7,order_repeat7,cart_repeat7,buy_ratio8,count_item8,count_user8,hour_mean8,day_mean8,repeat8,buy_ratio_std8,hour_std8,day_std8,prev8,next8,orders8,carts8,order_repeat8,cart_repeat8,buy_ratio9,count_item9,count_user9,hour_mean9,day_mean9,repeat9,buy_ratio_std9,hour_std9,day_std9,prev9,next9,orders9,carts9,order_repeat9,cart_repeat9,buy_ratio10,count_item10,unique_item10,hour_mean10,day_mean10,repeat10,buy_ratio_std10,hour_std10,day_std10,z10,z11,buy_ratio11,count_item11,count_user11,hour_mean11,day_mean11,repeat11,buy_ratio_std11,hour_std11,day_std11,buy_ratio12,count_item12,count_user12,hour_mean12,day_mean12,repeat12,buy_ratio_std12,hour_std12,day_std12,buy_ratio13,count_item13,count_user13,hour_mean13,day_mean13,repeat13,buy_ratio_std13,hour_std13,day_std13,buy_ratio14,count_item14,unique_item14,hour_mean14,day_mean14,repeat14,buy_ratio_std14,hour_std14,day_std14,buy_ratio15,count_item15,unique_item15,hour_mean15,day_mean15,repeat15,buy_ratio_std15,hour_std15,day_std15
0,12899779,59625,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.414214,-1.0,0.071773,-1.0,-1.0,1.414214,1.414214,1.414214,1.414214,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.000000,4.0,4.0,1.000000,0.000000,NaN,NaN,0.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.377964,0.377964,0.000000,12,11,56063.832031,315263.84375,1.090909,0.000000,18902.294922,174509.890625,0.000000,1.0,1.0,79200.000000,338400.0000,1.0,-1.000000,-1.000000,-1.000000,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,1,1,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,0,0,0,1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,0.000000,2.0,2.0,64435.000000,150835.00000,1.000000,0.00000,8051.117676,114136.937500,2.0,2.0,0.0,0.0,-1.0,-1.0,0.000000,6.0,5.0,66307.000000,325507.00000,1.200000,0.000000,7429.592773,169454.437500,5.0,5.0,0.0,0.0,-1.0,-1.0,0.000000,1.0,1.0,9485.000000,95885.00000,1.000000,-1.000000,-1.000000,-1.000000,1.0,1.0,0.0,0.0,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12899779,737445,1,3.0,1.0,-3.0,-3.0,-3.0,2.0,-3.0,0.850000,3.0,1.500000,8.0,8.0,-3.000000,-3.000000,-3.000000,0.850000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,2.0,1.0,0.022444,401.0,258.0,1.554264,0.148307,NaN,NaN,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,4.0,7.0,6.0,3.0,3.0,3.0,9.0,4.0,4.0,4.0,3.0,3.0,2.0,7.0,2.380476,2.380476,0.016234,924,575,50592.035156,303620.59375,1.606956,0.126442,19655.660156,170439.515625,0.028571,35.0,25.0,50927.429688,349624.5625,1.4,0.169031,21019.531250,161340.265625,15.0,12.0,0.0,1.0,-1.0,1.0,0.0,1,1,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,0,0,0,0.954863,-1.0,-1.0,-1.0,-1.0,-1.0,0.021201,283.0,180.0,51423.164062,294747.53125,1.572222,0.14431,19882.255

In [37]:
%%time
users.to_parquet(f'../../data/infer_with_features/train_v{VER}_LB_{PART}.pqt',index=False)

CPU times: user 7min 48s, sys: 15.7 s, total: 8min 4s
Wall time: 6min 6s
